In [1]:
import pandas as pd
import numpy as np
import functions as fc
import imlconnect
from datetime import datetime, timedelta
import time

In [2]:
print("Objectif du script :\n")
print("Détecter des écarts techno CPL (C G1 sur K G3 , C G3 sur K G1) qui n'ont pas de Dossier d'Analyse Ecart Techno CPL en cours. \n")
print("Entrez la date du jour svp ?\n")

j = input("Jour ? (format numérique svp!)\n")
while len(j)!=2:
    j = input("Jour ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val = int(str(j))
        while val<1 or val>31:
            j = input("Jour ? Entre 01 et 31 svp!)\n") 
            val = int(str(j))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        j = input("Jour ? (format numérique svp!)\n")
    else:
        print("Succès !\n")
        # No error; stop the loop
        break  

m = input("Mois ? (format numérique svp!)\n")
while len(m)!=2:
    m = input("Mois ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val1 = int(str(m))
        while val1<1 or val1>12:
            m = input("Mois ? Entre 01 et 12 svp!)\n") 
            val1 = int(str(m))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        m = input("Mois ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

a = input("Année en cours? (format numérique svp!)\n")
while len(a)!=4:
    a = input("Année ? (4 caractères et format numérique svp!)\n")
while True:
    try:
        val2 = int(str(a))
        while val2!=datetime.now().year:
            a = input("Année doit être année en cours svp!)\n") 
            val2 = int(str(a))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        a = input("Année ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

Objectif du script :

Détecter des écarts techno CPL (C G1 sur K G3 , C G3 sur K G1) qui n'ont pas de Dossier d'Analyse Ecart Techno CPL en cours. 

Entrez la date du jour svp ?



Jour ? (format numérique svp!)
 29


Succès !



Mois ? (format numérique svp!)
 12


Succès !



Année en cours? (format numérique svp!)
 2022


Succès !



In [3]:
date_saisie = pd.to_datetime(str(val2)+"-"+str(val1)+"-"+str(val)).strftime("%Y-%m-%d")
print("On va récupérer les données dans LinkyPilot en fonction de l'index date (J-1), vous devez retirer 1 jour à la date saisie.\n")
Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
while Q!='oui' and Q!='non':
    print("Ecrivez correctement oui ou non en minuscule svp!\n")
    Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
if Q=="oui":
    jour_à_retirer = input("Entrer le nombre de jour à retirer svp ?\n")
    while True:
        try:
            jour_à_retirer = int(str(jour_à_retirer))
        except ValueError:
            print("Ce n'est pas un nombre entier\n")
            jour_à_retirer = input("Entrez un nombre entier svp ?\n")
        else:
            print("Succès !\n")
            break
    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=jour_à_retirer)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
             
elif Q=='non':
    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=0)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
print("La date d'analyse est : \n",date_analyse) 
print("La date saisie est : \n",date_saisie)

On va récupérer les données dans LinkyPilot en fonction de l'index date (J-1), vous devez retirer 1 jour à la date saisie.



Voulez retirer des jours à la date d'analyse ? oui ou non ?
 oui
Entrer le nombre de jour à retirer svp ?
 1


Succès !

La date d'analyse est : 
 2022-12-28
La date saisie est : 
 2022-12-29


In [4]:
start = time.process_time()

In [5]:
chemin = "/var/projects/iml/rstudio/PPEO/11_Creation_DA_EcartTechnoCPL/"

# Nouvelle source plus récente à utiliser /  : https://dataviz.enedis.fr/t/LinkyPilot/views/HYP-REG-014-IncompatibilitesCetK/IncompatibilitsC-Kactuelles/
# 04/10/2022 : il faut maintenant télécharger en mode Tableau Croisé Excel le Inco J T , et convertir en CSV
# ================> renommer les en têtes du CSV en PDC_ID_C;ECART

# 1) Importer un fichier externe provenant d'un rapport LinkyPilot qui contient la liste des écarts techno
df_Liste_C = pd.read_csv(chemin+"src/ECART_TECHNO.csv", sep=';')


# Importer le fichier de maintenance Cube 2 colonnes au format CSV
# Ne garder dan l'onglet STOCK_SEMAINE que les colonnes reference_m et statut
# CSV et convertir en UTF8 tout court
df_MAINTENANCE_C = pd.read_csv(chemin+"src/maintenance.csv", sep=';')

In [6]:
my_conn_sup = imlconnect.Sql("lsup_conn", 
                  key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                  config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                  open_connection=True)

In [7]:
sql_ini = "select PDC_ID, PDC_ID_C, PDC_ID_PRM_CONSO,PDC_DATE_POSE,PDC_PDK_COM_FK, PDC_PDK_MERC_FK, PDC_STATUT_FK from lspmet.t_pdc inner join filter_list f on f.val = t_pdc.pdc_id_c where PDC_PDK_MERC_FK is not null and pdc_statut_fk=1"
#Fonction de creation complete du sql avec df = dataframe, value = Colonne du tableau utilisé pour filtre, sql_init = sql à executer  
sql = fc.create_sql_with_df(df_Liste_C,'PDC_ID_C',sql_ini)


In [8]:
# 2) Récupérer les infos de LinkySUP, uniquement si le PDC est à l'état Posé et si le PDK_MERC (PDK référent) n'est pas nul (indispensable pour détecter un écart techno)
df_Liste_PDC = my_conn_sup.query(sql)

In [9]:
df_Liste_PDC = df_Liste_PDC[df_Liste_PDC.pdc_date_pose <= datetime.now() - timedelta(days=8)]

In [10]:
# Recherche des DA Ecart Techno C (clos ou en cours)
df_DA_ECART_TECHNO_C = my_conn_sup.query("""select PDC_ID , PDC_ID_C,  DAN_NNI_OUVERTURE, DA.DAN_NUMERO_DOSSIER NUMERO_DA, statut_da.sda_libelle STATUT_DA 
                                                    from lspmet.t_dossier_analyse DA
                                                    join lspmet.T_ref_statut_dossier_analyse statut_da on DA.DAN_STATUT_DOSSIER_ANALYSE_FK = statut_da.sda_id
                                                    join lspmet.t_dysfonctionnement DYS on DYS.DYS_ID= DA.DAN_DYSFONCTIONNEMENT_FK
                                                    JOIN lspmet.T_REF_TYPE_DYSFONCTIONNEMENT TDY ON DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK = TDY.TDY_ID
                                                    JOIN lspmet.t_pdc on dys_id_objet = pdc_id
                                                    WHERE TDY_ID in (56)""")

In [11]:
# Recherche des DA Ecart Techno K (clos ou en cours)
df_DA_ECART_TECHNO_K = my_conn_sup.query("""select PDK_ID ,  DAN_NNI_OUVERTURE, DA.DAN_NUMERO_DOSSIER NUMERO_DA, statut_da.sda_libelle STATUT_DA 
                                            from lspmet.t_dossier_analyse DA
                                            join lspmet.T_ref_statut_dossier_analyse statut_da on DA.DAN_STATUT_DOSSIER_ANALYSE_FK = statut_da.sda_id
                                            join lspmet.t_dysfonctionnement DYS on DYS.DYS_ID= DA.DAN_DYSFONCTIONNEMENT_FK
                                            JOIN lspmet.T_REF_TYPE_DYSFONCTIONNEMENT TDY ON DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK = TDY.TDY_ID
                                            JOIN lspmet.t_pdk on dys_id_objet = pdk_id
                                            WHERE TDY_ID in (58)""")

In [12]:
""" Exemple d'agrégation : 
df_DA_ECART_TECHNO_C_STATS= df_DA_ECART_TECHNO_C.groupby(['pdc_id','pdc_id_c']).agg( 
                          # distinct count
                          nb_da_c = pd.NamedAgg(column ='pdc_id', aggfunc=lambda x: x.nunique()),
                          # count as event is unique
                          event_count = pd.NamedAgg(column ='event', aggfunc='count'),
                          # distinct count
                          name_count = pd.NamedAgg(column ='name', aggfunc=lambda x: x.nunique()),
                          # sum
                          sum_value = pd.NamedAgg(column ='value', aggfunc='sum'))"""

" Exemple d'agrégation : \ndf_DA_ECART_TECHNO_C_STATS= df_DA_ECART_TECHNO_C.groupby(['pdc_id','pdc_id_c']).agg( \n                          # distinct count\n                          nb_da_c = pd.NamedAgg(column ='pdc_id', aggfunc=lambda x: x.nunique()),\n                          # count as event is unique\n                          event_count = pd.NamedAgg(column ='event', aggfunc='count'),\n                          # distinct count\n                          name_count = pd.NamedAgg(column ='name', aggfunc=lambda x: x.nunique()),\n                          # sum\n                          sum_value = pd.NamedAgg(column ='value', aggfunc='sum'))"

In [13]:
df_DA_ECART_TECHNO_C_STATS = df_DA_ECART_TECHNO_C[df_DA_ECART_TECHNO_C.statut_da != 'Clos'].assign(nb_da_c = 1).groupby(by=['pdc_id','pdc_id_c'],as_index=False).nb_da_c.sum()

In [14]:
df_DA_ECART_TECHNO_K_STATS = df_DA_ECART_TECHNO_K[df_DA_ECART_TECHNO_K.statut_da != 'Clos'].assign(nb_da_k = 1).groupby(by='pdk_id',as_index=False).nb_da_k.sum()

In [15]:
df_Croisement = df_Liste_C.rename(columns = {'PDC_ID_C':'pdc_id_c'}).merge( df_Liste_PDC,on = 'pdc_id_c',how='inner').merge(df_DA_ECART_TECHNO_C_STATS.drop('pdc_id',axis=1),on= 'pdc_id_c',how='left').merge(df_DA_ECART_TECHNO_K_STATS.rename(columns = {'pdk_id':'pdc_pdk_merc_fk'}),on= 'pdc_pdk_merc_fk', how = 'left')

In [16]:
df_Croisement = df_Croisement[df_Croisement.nb_da_c.isnull() & df_Croisement.nb_da_k.isnull()]

In [17]:
# Vérifier qu'il n'y a pas une DIT en cours via un autre DA sur ces PDC
sql_ini1 = """select DAN_NUMERO_DOSSIER NUMERO_DA, DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK ID_DYS,
                                    DYS.DYS_ID_OBJET PDC, TDY.TDY_LIBELLE NOM_DA, statut_da.sda_libelle STATUT_DA,
                                    nvl(ACT_DATE_LANCEMENT,ACT_DATE_DEBUT) ACT_DATE_DEBUT, SAC_LIBELLE, TYA_LIBELLE, RAC_LIBELLE, SAC_ID 
                                    from lspmet.t_dysfonctionnement DYS 
                                    inner join filter_list f on f.val = DYS.DYS_ID_OBJET
                                    inner join lspmet.t_dossier_analyse DA on DA.DAN_DYSFONCTIONNEMENT_FK = DYS.DYS_ID
                                    JOIN lspmet.T_REF_TYPE_DYSFONCTIONNEMENT TDY ON DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK = TDY.TDY_ID
                                    join lspmet.T_PDC on DYS_ID_OBJET=PDC_ID
                                    join lspmet.T_ref_statut_dossier_analyse statut_da on DA.DAN_STATUT_DOSSIER_ANALYSE_FK = statut_da.sda_id
                                    join lspmet.t_localisation_dossier lod on lod.lod_dossier_fk = da.dan_id
                                    join lspmet.t_ref_localisation_erdf loc on loc.loc_id=lod.lod_localisation_fk
                                    left join lspmet.T_ACTION AC on AC.ACT_DOSSIER_ANALYSE_FK=DA.DAN_ID
                                    left join lspmet.T_REF_STATUT_ACTION  TSA on TSA.SAC_ID=AC.ACT_STATUT_ACTION_FK
                                    left join lspmet.T_TYPE_ACTION TTA on TTA.TYA_ID = AC.ACT_TYPE_ACTION_FK
                                    left join lspmet.T_RESULTAT_ACTION TRA on AC.ACT_ACTION_RESULTAT_FK = TRA.RAC_ID
                                    where DA.DAN_STATUT_DOSSIER_ANALYSE_FK<>3 AND ACT_ACTION_COURANTE=1"""
#Fonction de creation complete du sql avec df = dataframe, value = Colonne du tableau utilisé pour filtre, sql_init = sql à executer  
sql1 = fc.create_sql_with_df(df_Croisement,'pdc_id',sql_ini1)


In [18]:
df_Actions_C = my_conn_sup.query(sql1)

In [19]:
df_Action_Actuelle_C = df_Actions_C.query("tya_libelle =='T020/DIT C' and id_dys==18 or id_dys==47 or id_dys==61")

In [20]:
# .drop_duplicates() suprrime les doublons du dataframe en entier
df_Actions_C.query("tya_libelle =='T020/DIT C' and id_dys==18 or id_dys==47 or id_dys==61").drop_duplicates()

numero_da  id_dys             pdc  \
0    DA210628EWYRRP      18  PDC00002596042   
1    DA210628EWYRLX      18  PDC00021400758   
2    DA200304DTGJFR      61  PDC00003956770   
4    DA200226DSOWFF      18  PDC00023652567   
5    DA200427DVNBVI      18  PDC00017885013   
..              ...     ...             ...   
555  DA210201EPCEHA      18  PDC00028250417   
556  DA210329ESMDPC      18  PDC00034343142   
558  DA221010FLKABP      18  PDC00042626962   
561  DA220315FGIVDX      18  PDC00035474578   
562  DA221128FMHORT      18  PDC00042638959   

                                  nom_da statut_da      act_date_debut  \
0         Réco C - Absence de découverte  En cours 2021-07-07 12:15:40   
1         Réco C - Absence de découverte  En cours 2021-07-07 12:18:47   
2    C - Demande d'intervention en masse  En cours 2020-03-04 15:01:21   
4         Réco C - Absence de découverte  En cours 2021-01-05 15:14:37   
5         Réco C - Absence de découverte  En cours 2021-02-10 16:04:34   
..                                   ...       ...                 ...   
555       Réco C - Absence de découverte  En cours 2021-02-02 09:36:44   
556       Réco C - Absence de découverte  En cours 2021-10-21 15:43:18   
558       Réco C - Absence de découverte  En cours 2022-10-21 15:32:41   
561       Réco C - Absence de découverte  En cours 2022-03-16 11:45:26   
562       Réco C - Absence de découverte  En cours 2022-12-01 14:39:30   

    sac_libelle          tya_libelle rac_libelle  sac_id  
0    En attente           T020/DIT C        None       5  
1    En attente           T020/DIT C        None       5  
2    En attente  T045/DIT C en masse        None       5  
4    En attente           T020/DIT C        None       5  
5    En attente           T020/DIT C        None       5  
..          ...                  ...         ...     ...  
555  En attente           T020/DIT C        None       5  
556  En attente           T020/DIT C        None       5  
558  En attente           T020/DIT C        None       5  
561  En attente           T020/DIT C        None       5  
562  En attente           T020/DIT C        None       5  

[344 rows x 10 columns]

In [21]:
df_Action_Actuelle_C_racc = pd.DataFrame({'pdc':df_Action_Actuelle_C.pdc.unique(),'exclusion_c':'oui'})


In [22]:
# Vérifier qu'il n'y a pas une DIT en cours via un autre DA sur ces PDC
sql_ini2 = """select DAN_NUMERO_DOSSIER NUMERO_DA, DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK ID_DYS,DYS.DYS_ID_OBJET PDK, TDY.TDY_LIBELLE NOM_DA, statut_da.sda_libelle STATUT_DA, ACT_DATE_LANCEMENT ACT_DATE_DEBUT, SAC_LIBELLE, TYA_LIBELLE, RAC_LIBELLE, SAC_ID
                                    from lspmet.t_dysfonctionnement DYS 
                                    inner join filter_list f on f.val = DYS.DYS_ID_OBJET
                                    inner join lspmet.t_dossier_analyse DA on DA.DAN_DYSFONCTIONNEMENT_FK = DYS.DYS_ID
                                    JOIN lspmet.T_REF_TYPE_DYSFONCTIONNEMENT TDY ON DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK = TDY.TDY_ID
                                    join lspmet.T_PDK on DYS_ID_OBJET=PDK_ID
                                    join lspmet.T_ref_statut_dossier_analyse statut_da on DA.DAN_STATUT_DOSSIER_ANALYSE_FK = statut_da.sda_id
                                    join lspmet.t_localisation_dossier lod on lod.lod_dossier_fk = da.dan_id
                                    join lspmet.t_ref_localisation_erdf loc on loc.loc_id=lod.lod_localisation_fk
                                    left join lspmet.T_ACTION AC on AC.ACT_DOSSIER_ANALYSE_FK=DA.DAN_ID
                                    left join lspmet.T_REF_STATUT_ACTION  TSA on TSA.SAC_ID=AC.ACT_STATUT_ACTION_FK
                                    left join lspmet.T_TYPE_ACTION TTA on TTA.TYA_ID = AC.ACT_TYPE_ACTION_FK
                                    left join lspmet.T_RESULTAT_ACTION TRA on AC.ACT_ACTION_RESULTAT_FK = TRA.RAC_ID
                                    where DA.DAN_STATUT_DOSSIER_ANALYSE_FK<>3 and ACT_ACTION_COURANTE=1"""
#Fonction de creation complete du sql avec df = dataframe, value = Colonne du tableau utilisé pour filtre, sql_init = sql à executer  
sql2 = fc.create_sql_with_df(df_Croisement,'pdc_pdk_merc_fk',sql_ini2)


In [23]:
df_Actions_K = my_conn_sup.query(sql2)

In [24]:
df_Action_Actuelle_K = df_Actions_K[df_Actions_K.id_dys==58].assign(exclusion_k='oui')
df_Action_Actuelle_K_racc = pd.DataFrame({'pdk':df_Action_Actuelle_K.pdk.unique(),'exclusion_k':'oui'})


In [25]:
df_Analyse = df_Croisement.merge(df_Action_Actuelle_C_racc.rename(columns={'pdc':'pdc_id'}),on='pdc_id',how = 'left').merge(df_Action_Actuelle_K_racc.rename(columns = {'pdk':'pdc_pdk_merc_fk'}), on = 'pdc_pdk_merc_fk',how='left').replace(np.nan,0)

In [26]:
filters = [
    (df_Analyse.nb_da_c ==1) | (df_Analyse.nb_da_k ==1),
    (df_Analyse.nb_da_c ==0) & (df_Analyse.nb_da_k ==0) & (df_Analyse.exclusion_c =='oui'),
    (df_Analyse.nb_da_c ==0) & (df_Analyse.nb_da_k ==0) & (df_Analyse.exclusion_c ==0)
]
values = ['DA Ecart Techno C ou K en cours','Intervention en attente Autre DA','Pas de DA et aucune intervention Autre DA']
df_Analyse = df_Analyse.assign(categorie = np.select(filters, values))

In [27]:
df_Analyse = df_Analyse[df_Analyse.categorie == 'Pas de DA et aucune intervention Autre DA'].merge(df_MAINTENANCE_C.rename(columns={'reference_m':'pdc_id_prm_conso'}),on='pdc_id_prm_conso',how='left').replace(np.nan,'aucune')



In [28]:
df_Analyse_Finale_Creation_DA =df_Analyse[['pdc_id_c','ECART','pdc_id', 'pdc_id_prm_conso','statut']].groupby('pdc_id',as_index=False).agg(nb_statuts = pd.NamedAgg(column ='pdc_id', aggfunc='count'))

In [29]:
df_Analyse_Finale_Creation_DA = df_Analyse_Finale_Creation_DA.merge(df_Analyse,on='pdc_id',how='right')


In [30]:
x=df_Analyse_Finale_Creation_DA.groupby(['pdc_id', 'nb_statuts', 'pdc_id_c', 'ECART', 'pdc_id_prm_conso','statut'],as_index=False).agg(nb_affaires = pd.NamedAgg(column ='pdc_id', aggfunc='count'))


In [31]:
stats = x.pivot_table(index= 'pdc_id',columns='statut', values= 'nb_affaires')
stats = stats.reset_index().rename_axis(None, axis = 1).replace(np.nan,0)
stats = stats.assign(categorie_finale = np.where(stats.édité>0,'Présence Affaire Editée','Aucune Affaire Editée'))
stats = stats[~stats.pdc_id.isin(['PDC00010360283','PDC00010335879','PDC00015055238'])].query("categorie_finale=='Aucune Affaire Editée'")
print(stats)

             pdc_id  abandonné  aucune  terminé  à prendre en compte  édité  \
2    PDC00000296279        0.0     1.0      0.0                  0.0    0.0   
4    PDC00001539003        0.0     1.0      0.0                  0.0    0.0   
5    PDC00001928537        0.0     0.0      1.0                  0.0    0.0   
8    PDC00002720053        0.0     0.0      1.0                  0.0    0.0   
9    PDC00002762853        0.0     0.0      3.0                  0.0    0.0   
..              ...        ...     ...      ...                  ...    ...   
167  PDC00038767969        1.0     0.0      0.0                  0.0    0.0   
169  PDC00040674005        0.0     0.0      1.0                  0.0    0.0   
170  PDC00040695101        0.0     1.0      0.0                  0.0    0.0   
171  PDC00040714877        0.0     0.0      1.0                  0.0    0.0   
173  PDC00042708218        0.0     1.0      0.0                  0.0    0.0   

          categorie_finale  
2    Aucune Affaire Ed

In [32]:
# Faire un CSV par type d'écart 
df_DA_EcartTechnoCPL_CG1_KG3 = df_Analyse_Finale_Creation_DA[df_Analyse_Finale_Creation_DA.ECART.isin(['C P0 G1 - K P1 G3','C P1 G1 - K P1 G3'])][['pdc_id']].rename(columns={'pdc_id':'pdc'}).reset_index(drop=True)  


df_DA_EcartTechnoCPL_CG3_KG1 = df_Analyse_Finale_Creation_DA[df_Analyse_Finale_Creation_DA.ECART=='C P1 G3 - K P1 G1'][['pdc_id']].rename(columns={'pdc_id':'pdc'}).reset_index(drop=True)  


In [ ]:
import os 
directory = os.getcwd()
rep=directory+"/export_ecart_cpl/"

fc.split_df_to_csv(stats,999,rep,date_saisie,"stats")
fc.split_df_to_csv(df_DA_EcartTechnoCPL_CG1_KG3,999,rep,date_saisie,"DA_EcartTechnoCPL_CG1_KG3")
fc.split_df_to_csv(df_DA_EcartTechnoCPL_CG3_KG1,999,rep,date_saisie,"DA_EcartTechnoCPL_CG3_KG1")
# zip du dossier
import shutil as sh
sh.make_archive('export_ecart_cpl','zip',rep)